### Following actions should be performed:
 
- If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
- Check for null and unique values for test and train sets.
- Apply label encoder.
- Perform dimensionality reduction.
- Predict your test_df values using XGBoost.

In [45]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [46]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [47]:
df_train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [48]:
df_train.shape

(4209, 378)

In [49]:
df_train.describe()

,ID,y,X10,X11,X12,X13,X14,X15,X16,X17,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4209.000000,4209.000000,4209.000000,4209.0,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,...,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000
mean,4205.960798,100.669318,0.013305,0.0,0.075077,0.057971,0.428130,0.000475,0.002613,0.007603,...,0.318841,0.057258,0.314802,0.020670,0.009503,0.008078,0.007603,0.001663,0.000475,0.001426
std,2437.608688,12.679381,0.114590,0.0,0.263547,0.233716,0.494867,0.021796,0.051061,0.086872,...,0.466082,0.232363,0.464492,0.142294,0.097033,0.089524,0.086872,0.040752,0.021796,0.037734
min,0.000000,72.110000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2095.000000,90.820000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4220.000000,99.150000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6314.000000,109.010000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8417.000000,265.320000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
df_train.columns

Index(['ID', 'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=378)

In [51]:
df_train.info

<bound method DataFrame.info of         ID       y  X0 X1  X2 X3 X4  X5 X6 X8  ...  X375  X376  X377  X378  \
0        0  130.81   k  v  at  a  d   u  j  o  ...     0     0     1     0   
1        6   88.53   k  t  av  e  d   y  l  o  ...     1     0     0     0   
2        7   76.26  az  w   n  c  d   x  j  x  ...     0     0     0     0   
3        9   80.62  az  t   n  f  d   x  l  e  ...     0     0     0     0   
4       13   78.02  az  v   n  f  d   h  d  n  ...     0     0     0     0   
...    ...     ...  .. ..  .. .. ..  .. .. ..  ...   ...   ...   ...   ...   
4204  8405  107.39  ak  s  as  c  d  aa  d  q  ...     1     0     0     0   
4205  8406  108.77   j  o   t  d  d  aa  h  h  ...     0     1     0     0   
4206  8412  109.22  ak  v   r  a  d  aa  g  e  ...     0     0     1     0   
4207  8415   87.48  al  r   e  f  d  aa  l  u  ...     0     0     0     0   
4208  8417  110.85   z  r  ae  c  d  aa  g  w  ...     1     0     0     0   

      X379  X380  X382  X383  X

## If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
- Seperating x_train and y_train dataset

In [52]:
xtrain=df_train.drop('y',axis=1)
ytrain=df_train['y']

In [53]:
xtrain.shape

(4209, 377)

In [54]:
ytrain.shape

(4209,)

In [55]:
xtrain.var()

ID      5.941936e+06
X10     1.313092e-02
X11     0.000000e+00
X12     6.945713e-02
X13     5.462335e-02
            ...     
X380    8.014579e-03
X382    7.546747e-03
X383    1.660732e-03
X384    4.750593e-04
X385    1.423823e-03
Length: 369, dtype: float64

In [56]:
df_test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [57]:
df_test.shape

(4209, 377)

In [58]:
df_test.describe()

,ID,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,...,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000
mean,4211.039202,0.019007,0.000238,0.074364,0.061060,0.427893,0.000713,0.002613,0.008791,0.010216,...,0.325968,0.049656,0.311951,0.019244,0.011879,0.008078,0.008791,0.000475,0.000713,0.001663
std,2423.078926,0.136565,0.015414,0.262394,0.239468,0.494832,0.026691,0.051061,0.093357,0.100570,...,0.468791,0.217258,0.463345,0.137399,0.108356,0.089524,0.093357,0.021796,0.026691,0.040752
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2115.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4202.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6310.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8416.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [59]:
df_test.info

<bound method DataFrame.info of         ID  X0  X1  X2 X3 X4  X5 X6 X8  X10  ...  X375  X376  X377  X378  \
0        1  az   v   n  f  d   t  a  w    0  ...     0     0     0     1   
1        2   t   b  ai  a  d   b  g  y    0  ...     0     0     1     0   
2        3  az   v  as  f  d   a  j  j    0  ...     0     0     0     1   
3        4  az   l   n  f  d   z  l  n    0  ...     0     0     0     1   
4        5   w   s  as  c  d   y  i  m    0  ...     1     0     0     0   
...    ...  ..  ..  .. .. ..  .. .. ..  ...  ...   ...   ...   ...   ...   
4204  8410  aj   h  as  f  d  aa  j  e    0  ...     0     0     0     0   
4205  8411   t  aa  ai  d  d  aa  j  y    0  ...     0     1     0     0   
4206  8413   y   v  as  f  d  aa  d  w    0  ...     0     0     0     0   
4207  8414  ak   v  as  a  d  aa  c  q    0  ...     0     0     1     0   
4208  8416   t  aa  ai  c  d  aa  g  r    0  ...     1     0     0     0   

      X379  X380  X382  X383  X384  X385  
0        0  

In [60]:
df_test.columns

Index(['ID', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'X10',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=377)

In [61]:
for i in xtrain:
    if (xtrain[i].dtype=='O'):
            continue
    elif(xtrain.var()[i]==0):
        xtrain.drop(i,axis=1,inplace=True)
        df_test.drop(i,axis=1,inplace=True)
          

In [63]:
xtrain.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,6,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,7,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,9,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,13,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
xtrain.shape

(4209, 365)

## Check for null and unique values for test and train sets.


In [68]:
xtrain.isnull().sum()

ID      0
X0      0
X1      0
X2      0
X3      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 365, dtype: int64

In [74]:
for i in xtrain:
    if(xtrain[i].isnull().sum!=0):
        print(i)

ID
X0
X1
X2
X3
X4
X5
X6
X8
X10
X12
X13
X14
X15
X16
X17
X18
X19
X20
X21
X22
X23
X24
X26
X27
X28
X29
X30
X31
X32
X33
X34
X35
X36
X37
X38
X39
X40
X41
X42
X43
X44
X45
X46
X47
X48
X49
X50
X51
X52
X53
X54
X55
X56
X57
X58
X59
X60
X61
X62
X63
X64
X65
X66
X67
X68
X69
X70
X71
X73
X74
X75
X76
X77
X78
X79
X80
X81
X82
X83
X84
X85
X86
X87
X88
X89
X90
X91
X92
X94
X95
X96
X97
X98
X99
X100
X101
X102
X103
X104
X105
X106
X108
X109
X110
X111
X112
X113
X114
X115
X116
X117
X118
X119
X120
X122
X123
X124
X125
X126
X127
X128
X129
X130
X131
X132
X133
X134
X135
X136
X137
X138
X139
X140
X141
X142
X143
X144
X145
X146
X147
X148
X150
X151
X152
X153
X154
X155
X156
X157
X158
X159
X160
X161
X162
X163
X164
X165
X166
X167
X168
X169
X170
X171
X172
X173
X174
X175
X176
X177
X178
X179
X180
X181
X182
X183
X184
X185
X186
X187
X189
X190
X191
X192
X194
X195
X196
X197
X198
X199
X200
X201
X202
X203
X204
X205
X206
X207
X208
X209
X210
X211
X212
X213
X214
X215
X216
X217
X218
X219
X220
X221
X222
X223
X224
X225
X226
X227
X228
X229
X230

In [73]:
for i in df_test:
    if(df_test[i].isnull().sum()!=0):
        print(i)

In [75]:
ytrain.isnull().sum()

0

In [76]:
for i in xtrain:
    if (xtrain[i].dtype=='O'):
        print(i)
        print(xtrain[i].unique())

X0
['k' 'az' 't' 'al' 'o' 'w' 'j' 'h' 's' 'n' 'ay' 'f' 'x' 'y' 'aj' 'ak' 'am'
 'z' 'q' 'at' 'ap' 'v' 'af' 'a' 'e' 'ai' 'd' 'aq' 'c' 'aa' 'ba' 'as' 'i'
 'r' 'b' 'ax' 'bc' 'u' 'ad' 'au' 'm' 'l' 'aw' 'ao' 'ac' 'g' 'ab']
X1
['v' 't' 'w' 'b' 'r' 'l' 's' 'aa' 'c' 'a' 'e' 'h' 'z' 'j' 'o' 'u' 'p' 'n'
 'i' 'y' 'd' 'f' 'm' 'k' 'g' 'q' 'ab']
X2
['at' 'av' 'n' 'e' 'as' 'aq' 'r' 'ai' 'ak' 'm' 'a' 'k' 'ae' 's' 'f' 'd'
 'ag' 'ay' 'ac' 'ap' 'g' 'i' 'aw' 'y' 'b' 'ao' 'al' 'h' 'x' 'au' 't' 'an'
 'z' 'ah' 'p' 'am' 'j' 'q' 'af' 'l' 'aa' 'c' 'o' 'ar']
X3
['a' 'e' 'c' 'f' 'd' 'b' 'g']
X4
['d' 'b' 'c' 'a']
X5
['u' 'y' 'x' 'h' 'g' 'f' 'j' 'i' 'd' 'c' 'af' 'ag' 'ab' 'ac' 'ad' 'ae'
 'ah' 'l' 'k' 'n' 'm' 'p' 'q' 's' 'r' 'v' 'w' 'o' 'aa']
X6
['j' 'l' 'd' 'h' 'i' 'a' 'g' 'c' 'k' 'e' 'f' 'b']
X8
['o' 'x' 'e' 'n' 's' 'a' 'h' 'p' 'm' 'k' 'd' 'i' 'v' 'j' 'b' 'q' 'w' 'g'
 'y' 'l' 'f' 'u' 'r' 't' 'c']


In [77]:
for i in df_test:
    if (df_test[i].dtype=='O'):
        print(i)
        print(df_test[i].unique())

X0
['az' 't' 'w' 'y' 'x' 'f' 'ap' 'o' 'ay' 'al' 'h' 'z' 'aj' 'd' 'v' 'ak'
 'ba' 'n' 'j' 's' 'af' 'ax' 'at' 'aq' 'av' 'm' 'k' 'a' 'e' 'ai' 'i' 'ag'
 'b' 'am' 'aw' 'as' 'r' 'ao' 'u' 'l' 'c' 'ad' 'au' 'bc' 'g' 'an' 'ae' 'p'
 'bb']
X1
['v' 'b' 'l' 's' 'aa' 'r' 'a' 'i' 'p' 'c' 'o' 'm' 'z' 'e' 'h' 'w' 'g' 'k'
 'y' 't' 'u' 'd' 'j' 'q' 'n' 'f' 'ab']
X2
['n' 'ai' 'as' 'ae' 's' 'b' 'e' 'ak' 'm' 'a' 'aq' 'ag' 'r' 'k' 'aj' 'ay'
 'ao' 'an' 'ac' 'af' 'ax' 'h' 'i' 'f' 'ap' 'p' 'au' 't' 'z' 'y' 'aw' 'd'
 'at' 'g' 'am' 'j' 'x' 'ab' 'w' 'q' 'ah' 'ad' 'al' 'av' 'u']
X3
['f' 'a' 'c' 'e' 'd' 'g' 'b']
X4
['d' 'b' 'a' 'c']
X5
['t' 'b' 'a' 'z' 'y' 'x' 'h' 'g' 'f' 'j' 'i' 'd' 'c' 'af' 'ag' 'ab' 'ac'
 'ad' 'ae' 'ah' 'l' 'k' 'n' 'm' 'p' 'q' 's' 'r' 'v' 'w' 'o' 'aa']
X6
['a' 'g' 'j' 'l' 'i' 'd' 'f' 'h' 'c' 'k' 'e' 'b']
X8
['w' 'y' 'j' 'n' 'm' 's' 'a' 'v' 'r' 'o' 't' 'h' 'c' 'k' 'p' 'u' 'd' 'g'
 'b' 'q' 'e' 'l' 'f' 'i' 'x']


## Apply Label Encoder.


In [78]:
from sklearn import preprocessing

In [80]:
label_encoder = preprocessing.LabelEncoder()

In [81]:
for i in xtrain:
    if (xtrain[i].dtype!='int64'):
        xtrain[i]= label_encoder.fit_transform(xtrain[i])
        print(i)
        print(xtrain[i].unique())

X0
[32 20 40  9 36 43 31 29 39 35 19 27 44 45  7  8 10 46 37 15 12 42  5  0
 26  6 25 13 24  1 22 14 30 38 21 18 23 41  4 16 34 33 17 11  3 28  2]
X1
[23 21 24  3 19 13 20  1  4  0  6  9 26 11 16 22 17 15 10 25  5  7 14 12
  8 18  2]
X2
[17 19 34 25 16 14 38  7  8 33  0 31  3 39 26 24  5 21  2 13 27 29 20 42
 22 12  9 28 41 18 40 11 43  6 36 10 30 37  4 32  1 23 35 15]
X3
[0 4 2 5 3 1 6]
X4
[3 1 2 0]
X5
[24 28 27 12 11 10 14 13  9  8  5  6  1  2  3  4  7 16 15 18 17 20 21 23
 22 25 26 19  0]
X6
[ 9 11  3  7  8  0  6  2 10  4  5  1]
X8
[14 23  4 13 18  0  7 15 12 10  3  8 21  9  1 16 22  6 24 11  5 20 17 19
  2]


In [82]:
for i in df_test:
    if (df_test[i].dtype!='int64'):
        df_test[i]= label_encoder.fit_transform(df_test[i])
        print(i)
        print(df_test[i].unique())

X0
[21 42 45 47 46 29 12 38 20  8 31 48  6 27 44  7 23 37 33 41  3 19 15 13
 17 36 34  0 28  5 32  4 22  9 18 14 40 11 43 35 26  1 16 25 30 10  2 39
 24]
X1
[23  3 13 20  1 19  0 10 17  4 16 14 26  6  9 24  8 12 25 21 22  5 11 18
 15  7  2]
X2
[34  8 17  4 38 24 26 10 33  0 16  6 37 32  9 23 14 13  2  5 22 29 30 27
 15 35 19 39 44 43 21 25 18 28 12 31 42  1 41 36  7  3 11 20 40]
X3
[5 0 2 4 3 6 1]
X4
[3 1 0 2]
X5
[26  9  0 31 30 29 14 13 12 16 15 11 10  6  7  2  3  4  5  8 18 17 20 19
 22 23 25 24 27 28 21  1]
X6
[ 0  6  9 11  8  3  5  7  2 10  4  1]
X8
[22 24  9 13 12 18  0 21 17 14 19  7  2 10 15 20  3  6  1 16  4 11  5  8
 23]


## Perform dimensionality reduction.


In [83]:
std_scalar=preprocessing.StandardScaler()

In [84]:
xtrain = std_scalar.fit_transform(xtrain)

In [85]:
df_test = std_scalar.fit_transform(df_test)

In [86]:
from sklearn.decomposition import PCA

In [87]:
pca=PCA()

In [88]:
xtrain_pca=pca.fit_transform(xtrain)

In [89]:
df_test_pca=pca.fit_transform(df_test)

In [90]:
pca.explained_variance_ratio_

array([7.12486653e-02, 5.62415084e-02, 4.79979004e-02, 3.48818222e-02,
       3.26672655e-02, 3.17006619e-02, 2.82351002e-02, 2.14035131e-02,
       1.91298040e-02, 1.74214677e-02, 1.66286781e-02, 1.64393781e-02,
       1.49384640e-02, 1.38777649e-02, 1.35286203e-02, 1.27571090e-02,
       1.22387389e-02, 1.17690111e-02, 1.10581757e-02, 1.06911106e-02,
       1.02639043e-02, 9.37624352e-03, 9.29049134e-03, 9.05267345e-03,
       8.59958787e-03, 8.54160869e-03, 7.82044339e-03, 7.48747967e-03,
       7.34403625e-03, 7.23314434e-03, 7.03553099e-03, 6.87577048e-03,
       6.75483979e-03, 6.50479088e-03, 6.37950579e-03, 6.19292941e-03,
       6.13882865e-03, 6.03783025e-03, 5.99492525e-03, 5.89027248e-03,
       5.55576539e-03, 5.48572168e-03, 5.28121707e-03, 5.25910912e-03,
       5.12864286e-03, 5.02484447e-03, 4.94931135e-03, 4.76961699e-03,
       4.71278343e-03, 4.63198213e-03, 4.55830740e-03, 4.47537732e-03,
       4.35658769e-03, 4.28679476e-03, 4.22548636e-03, 4.04582798e-03,
      

In [92]:
pca=PCA(n_components=200)

xtrain_pca=pca.fit_transform(xtrain)

df_test_pca=pca.fit_transform(df_test)

pca.explained_variance_ratio_

array([0.07124867, 0.05624151, 0.0479979 , 0.03488182, 0.03266727,
       0.03170066, 0.0282351 , 0.02140351, 0.0191298 , 0.01742147,
       0.01662868, 0.01643938, 0.01493846, 0.01387776, 0.01352862,
       0.01275711, 0.01223874, 0.01176901, 0.01105818, 0.01069111,
       0.0102639 , 0.00937624, 0.00929049, 0.00905267, 0.00859959,
       0.00854161, 0.00782044, 0.00748748, 0.00734404, 0.00723314,
       0.00703553, 0.00687577, 0.00675484, 0.00650479, 0.00637951,
       0.00619293, 0.00613883, 0.00603783, 0.00599493, 0.00589027,
       0.00555577, 0.00548572, 0.00528122, 0.00525911, 0.00512864,
       0.00502484, 0.00494931, 0.00476962, 0.00471278, 0.00463198,
       0.00455831, 0.00447538, 0.00435659, 0.00428679, 0.00422549,
       0.00404583, 0.00400033, 0.00393454, 0.00390301, 0.00381968,
       0.00380986, 0.00372237, 0.00365473, 0.00364321, 0.0036187 ,
       0.00355727, 0.00346129, 0.00341047, 0.00337771, 0.00336256,
       0.00331947, 0.00330331, 0.00327653, 0.00322861, 0.00319

## Predict your test_df values using XGBoost.


In [93]:
import xgboost as xgb

In [94]:
xgb_model=xgb.XGBRegressor(objective ='reg:linear', n_estimators = 10)


In [95]:
xgb_model.fit(xtrain_pca,ytrain)

[21:40:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=4,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [96]:
xgb_model.predict(df_test_pca)

array([ 78.506714,  90.84197 ,  80.794586, ...,  98.53473 , 103.381874,
        90.35204 ], dtype=float32)

In [98]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [100]:
ytrain_pred_xgb2=xgb_model.predict((xtrain_pca))
np.sqrt(mean_squared_error(ytrain ,ytrain_pred_xgb2))

7.7529378889435625

In [102]:
r2_score(ytrain ,ytrain_pred_xgb2)


0.6260274887750187

In [103]:
pd.DataFrame({"Actual Train Values" : ytrain , "Model Predictions" :  ytrain_pred_xgb2})

,Actual Train Values,Model Predictions
0,130.81,111.508102
1,88.53,89.199455
2,76.26,77.150223
3,80.62,81.635017
4,78.02,78.241142
...,...,...
4204,107.39,102.831642
4205,108.77,104.847939
4206,109.22,107.898170
4207,87.48,89.820564


In [104]:
#####################################################